In [ ]:
import os
import shutil
import zipfile

global_packages = '\\\\pacman\packages'
dir_workspace = os.path.join(os.getcwd(),'workspace_alexa')
dir_release = 'E:\Release'
specific_package = None
#specific_package = 'D:\DUMP\JK\Installation_Studio_8.8.0.004.zip'

In [ ]:
def get_build_package(pkg_name,major_version,version):
    dir_packages = global_packages+os.path.sep+pkg_name
    dict_version_name = {f.replace(major_version+'_',''):f for f in os.listdir(dir_packages) if major_version in f}
    latest_pkg_version = dict_version_name[max(dict_version_name.keys())] if version == None else dict_version_name[version]
    latest_pkg_dir = os.path.join(dir_packages,latest_pkg_version+os.path.sep+'00001')
    latest_pkg = [os.path.join(latest_pkg_dir,f) for f in os.listdir(latest_pkg_dir)][0]
    return latest_pkg

In [ ]:
def configure_connection(dir_install,major_version,config_name):
    file_conn = os.path.join(dir_workspace,major_version+'_'+config_name)
    shutil.copy(file_conn,os.path.join(dir_install,'workspace'))
    file_conn_target = os.path.join(os.path.join(dir_install,'workspace'),major_version+'_'+config_name)
    file_conn_target_ren = os.path.join(os.path.join(dir_install,'workspace'),config_name)
    os.rename(file_conn_target,file_conn_target_ren)
    return

In [ ]:
def do_install_latest_package(package_name,install_name,major_version,version):
    if(specific_package != None and package_name in specific_package):
        file_package = specific_package
    else:
        file_package = get_build_package(package_name,major_version,version)
    
    # Package Extraction
    dir_extract = os.path.join(dir_workspace,'extract')
    if os.path.exists(dir_extract) and os.path.isdir(dir_extract):
        shutil.rmtree(dir_extract)
    os.mkdir(dir_extract) 
    
    shutil.copy(file_package,dir_extract)
    
    file_package_target = os.path.join(dir_extract,os.path.basename(file_package))
    with zipfile.ZipFile(file_package_target, 'r') as zip_ref:
        zip_ref.extractall(dir_extract)
          
    # Maintain Old Installation
    dir_release_version = os.path.join(dir_release,major_version)
    dir_install_old = os.path.join(dir_release_version,install_name+'_old')
    if os.path.exists(dir_install_old) and os.path.isdir(dir_install_old):
        shutil.rmtree(dir_install_old)
    dir_install = os.path.join(dir_release_version,install_name)
    if os.path.exists(dir_install) and os.path.isdir(dir_install):
        os.rename(dir_install,dir_install_old)
    
    #Installation
    file_package_in = os.path.join(dir_extract,install_name+'.zip')
    with zipfile.ZipFile(file_package_in, 'r') as zip_ref:
        zip_ref.extractall(dir_install)
    os.mkdir(os.path.join(dir_install,'workspace'))
    shutil.copy(os.path.join(dir_workspace,'ojdbc8.jar'),os.path.join(dir_install,'externals'))
        
    print('Successfully installed '+os.path.basename(file_package))
    return dir_install

In [ ]:
def do_install_mapper(major_version,version=None):
    dir_install = do_install_latest_package('GSO_Mapper','GSOMapper',major_version,version)
    configure_connection(dir_install,major_version,'connectionDetail.xml')
    return

In [ ]:
def do_install_istudio(major_version,version=None):
    dir_install = do_install_latest_package('Installation_Studio','InstallationStudio',major_version,version)
    configure_connection(dir_install,major_version,'RepositoryConnectionProfiles.xml')
    return

In [5]:
from lxml import etree as ET
import os
import yaml
dir_workspace = os.path.join(os.getcwd(),'workspace_alexa')

# method to get path of element
def iterate_for_path(elm,path):
    path = '.'+elm.attrib['name']+path
    if (elm.getparent() != None and 'name' in elm.getparent().attrib):
        return iterate_for_path(elm.getparent(),path)
    return path

# Generate path : multiplicity dictinary
file_name_cs = os.path.join(dir_workspace,'mapping.meta')
tree_meta = ET.parse(file_name_cs)
list_elm_part = [elm_part for elm_part in tree_meta.findall('.//participants') if elm_part.attrib['type'] == 'Field']
dict_data = {iterate_for_path(elm,'')[1:]:elm.attrib['multiple'] == 'true' for elm in list_elm_part}

# Dump multiplicity meta in yaml
with open(os.path.join(dir_workspace,'data.yml'), 'w') as outfile:
    yaml.dump(dict_data, outfile, default_flow_style=False)
    
print(len(dict_data))

89990


In [ ]:
def test_path(path):
    path = 'xxx'
    return path
p = test_path('abc')
p